### import dataset from kaggle

In [ ]:
! pip install opendatasets

In [ ]:
import opendatasets

opendatasets.download("https://www.kaggle.com/datasets/kaiska/apparel-dataset/download?datasetVersionNumber=2")


### FashionNet architecture

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Lambda, Dense, Flatten, Input
import tensorflow as tf



class FashionNet:

  @staticmethod
  def build_category_branch(inputs, numCategories, finalAct="softmax", chanDim=-1):

    # utilize a lambda layer to convert the 3 channel input to a grayscale representation
    x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)

    # CONV => RELU => POOL
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Dropout(0.25)(x)

    # (CONV => RELU) * 2 => POOL
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # (CONV => RELU) * 2 => POOL
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # define a branch of output layers for the number of different clothing categories (i.e., shirts, jeans, dresses, etc.)
    x = Flatten()(x)
    x = Dense(256)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # The last activation layer is fully connected and has the same number of neurons/outputs as our numCategories
    x = Dense(numCategories)(x)
    x = Activation(finalAct, name="category_output")(x)

    return x


  @staticmethod
  def build_color_branch(inputs, numColors, finalAct="softmax", chanDim=-1):

    # CONV => RELU => POOL
    x = Conv2D(16, (3, 3), padding="same")(inputs)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Dropout(0.25)(x)

    # CONV => RELU => POOL
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # CONV => RELU => POOL
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # define a branch of output layers for the number of different colors (i.e., red, black, blue, etc.)
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # the final activation layer for the color branch
    x = Dense(numColors)(x)
    x = Activation(finalAct, name="color_output")(x)

    return x


  @staticmethod
  def build(width, height, numCategories, numColors, finalAct="softmax"):
    inputShape = (height, width, 3)
    chanDim = -1

    # construct both the "category" and "color" sub-networks
    inputs = Input(shape=inputShape)    # Input(shape=None,batch_size=None,name=None,dtype=None,sparse=None,tensor=None,ragged=None,type_spec=None,**kwargs)

    categoryBranch = FashionNet.build_category_branch(inputs,
      numCategories, finalAct=finalAct, chanDim=chanDim)

    colorBranch = FashionNet.build_color_branch(inputs,
      numColors, finalAct=finalAct, chanDim=chanDim)


    # create the model using our input (the batch of images) and
    # two separate outputs -- one for the clothing category branch and another for the color branch, respectively

    model = Model(inputs=inputs, outputs=[categoryBranch, colorBranch], name="fashionnet")

    # return the constructed network architecture
    return model


### preprocess

In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os





# initialize the number of epochs to train for, initial learning rate, batch size, and image dimensions
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)


'''
image path another method:
l = []
for root, dirs, files in os.walk('/content/apparel-dataset'):
  for file in files:
    l.append(os.path.join(root, file))

print(l)
'''

imagePaths = sorted(list(paths.list_images("/content/apparel-dataset")))
random.seed(42)
random.shuffle(imagePaths)

# initialize the data, clothing category labels (i.e., shirts, jeans,
# dresses, etc.) along with the color labels (i.e., red, blue, etc.)
data = []
categoryLabels = []
colorLabels = []

# loop over the input images
for imagePath in imagePaths:
  # load the image, pre-process it, and store it in the data list
  image = cv2.imread(imagePath)
  image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = img_to_array(image)
  data.append(image)

  # extract the clothing color and category from the path and update the respective lists
  (color, cat) = imagePath.split(os.path.sep)[-2].split("_")
  categoryLabels.append(cat)
  colorLabels.append(color)


# scale the raw pixel intensities to the range [0, 1] and convert to a NumPy array
data = np.array(data, dtype="float") / 255.0

# convert the label lists to NumPy arrays prior to binarization
categoryLabels = np.array(categoryLabels)
colorLabels = np.array(colorLabels)

# binarize both sets of labels
categoryLB = LabelBinarizer()
colorLB = LabelBinarizer()
categoryLabels = categoryLB.fit_transform(categoryLabels)
colorLabels = colorLB.fit_transform(colorLabels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainCategoryY, testCategoryY, trainColorY, testColorY) = train_test_split(data, categoryLabels, colorLabels, test_size=0.2, random_state=42)





### compile and train

In [ ]:
# initialize our FashionNet multi-output network
model = FashionNet().build(96, 96, numCategories= len(categoryLB.classes_), numColors=len(colorLB.classes_), finalAct="softmax")

# define two dictionaries: one that specifies the loss method for
# each output of the network along with a second dictionary that
# specifies the weight per loss
losses = {
    "category_output": "categorical_crossentropy",
    "color_output": "categorical_crossentropy"}


# loss_weights allows you to specify the contribution of each loss function to the total training loss.
# This is useful for multi-task learning models where you want to balance multiple objectives.
lossWeights = {"category_output": 1.0, "color_output": 1.0}

# initialize the optimizer and compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=losses, loss_weights=lossWeights, metrics=["accuracy"])


H = model.fit(x = trainX,
              y = {"category_output": trainCategoryY, "color_output": trainColorY},
              validation_data=(testX, {"category_output": testCategoryY, "color_output": testColorY}),
              epochs=EPOCHS,
              verbose=1)


Epoch 1/50
405/405 [==============================] - ETA: 0s - loss: 1.8155 - category_output_loss: 0.9815 - color_output_loss: 0.8340 - category_output_accuracy: 0.6955 - color_output_accuracy: 0.7299

In [ ]:
model.summery()

### save model and labels

In [ ]:
model.save("Multiple_outputs_and_multiple_losses.hdf5")

# save the category binarizer to disk
with open("categoryLabels", 'wb') as f:
  pickle.dump(categoryLB,f)

# save the color binarizer to disk
with open("categoryLabels", 'wb') as b:
  pickle.dump(colorLB,b)



### plot

In [ ]:
# plot the total loss, category loss, and color loss
lossNames = ["loss", "category_output_loss", "color_output_loss"]
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))

# loop over the loss names
for (i, l) in enumerate(lossNames):
  # plot the loss for both the training and validation data
  title = "Loss for {}".format(l) if l != "loss" else "Total loss"
  ax[i].set_title(title)
  ax[i].set_xlabel("Epoch #")
  ax[i].set_ylabel("Loss")
  ax[i].plot(np.arange(0, EPOCHS), H.history[l], label=l)
  ax[i].plot(np.arange(0, EPOCHS), H.history["val_" + l],
    label="val_" + l)
  ax[i].legend()

# save the losses figure
plt.tight_layout()
plt.savefig("losses.png")
plt.close()


In [ ]:
# create a new figure for the accuracies
accuracyNames = ["category_output_accuracy", "color_output_accuracy"]
plt.style.use("ggplot")
(fig, ax) = plt.subplots(2, 1, figsize=(8, 8))
# loop over the accuracy names
for (i, l) in enumerate(accuracyNames):
  # plot the loss for both the training and validation data
  ax[i].set_title("Accuracy for {}".format(l))
  ax[i].set_xlabel("Epoch #")
  ax[i].set_ylabel("Accuracy")
  ax[i].plot(np.arange(0, EPOCHS), H.history[l], label=l)
  ax[i].plot(np.arange(0, EPOCHS), H.history["val_" + l],
    label="val_" + l)
  ax[i].legend()
# save the accuracies figure
plt.tight_layout()
plt.savefig("accuracies.png")
plt.close()

### load model and test image

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import argparse
import imutils
import pickle
import cv2


# load the image
image = cv2.imread("/kaggle/input/img-test/pants-thm-02c.jpg")
output = imutils.resize(image, width=400)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# load the trained convolutional neural network from disk, followed
# by the category and color label binarizers, respectively

model = load_model("/kaggle/working/Multiple_outputs_and_multiple_losses.hdf5")

with open("/kaggle/working/categoryLabels", "rb") as f:
    categoryLB = pickle.load(f)

with open("/kaggle/working/colorLabels", "rb") as s:
    colorLB = pickle.load(s)



# classify the input image using Keras' multi-output functionality
(categoryProba, colorProba) = model.predict(image)

# find indexes of both the category and color outputs with the
# largest probabilities, then determine the corresponding class labels
categoryIdx = categoryProba[0].argmax()
colorIdx = colorProba[0].argmax()
categoryLabel = categoryLB.classes_[categoryIdx]
colorLabel = colorLB.classes_[colorIdx]

In [ ]:
print(categoryLabel)
print(colorLabel)